In [1]:
import torch
import sys
from datasets import load_dataset

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, PLAYWRIGHT_CONFIG, SMOL_LM_135M

/home/ubuntu/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = PLAYWRIGHT_CONFIG
cfg.notebook_mode = True
cfg.model_id_or_path = SMOL_LM_135M
wrapper = TrainerWrapper(cfg)

2025-01-21 14:33:18.385 | INFO     | trl_wrapper.trainer_wrapper:__init__:212 - Adding special tokens: ['<thought>', '</thought>', '<solution>', '</solution>']


In [ ]:
cfg.data_module_choice

'playwright_summary_to_script'

In [4]:
wrapper.init_model()

2025-01-21 14:34:46.894 | INFO     | trl_wrapper.trainer_wrapper:init_model:230 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: flash_attention_2


In [6]:
wrapper.config.tuning_mode

'sft'

In [9]:
wrapper.init_data_module()

2025-01-21 14:51:47.703 | INFO     | model.utils:__init__:115 - Cache dir: ../dataset_caches/playwright_summary_to_script
2025-01-21 14:51:47.704 | INFO     | model.utils:setup:156 - Loading dataset for stage fit
2025-01-21 14:51:47.705 | INFO     | model.utils:setup:159 - Processing dataset for stage fit, workers: 1, cache dir ../dataset_caches/playwright_summary_to_script, using cache: False
2025-01-21 14:51:47.705 | INFO     | model.utils:init_dataset:131 - Loading local dataset from ../screenplay_scenes_summarized.parquet


2025-01-21 14:51:47.733 | INFO     | model.utils:init_dataset:151 - Loading custom chat template: ../chat_templates/ministral_8b.jinja
2025-01-21 14:51:47.734 | INFO     | model.utils:setup:167 - Train dataset size: 9604 Val dataset size: 1068
Map: 100%|██████████| 1068/1068 [00:00<00:00, 71137.31 examples/s]


In [ ]:
# Tokenizer dev
first_batch = wrapper.data_module.train_dataset[0]
tokenized = wrapper.data_module._tokenize_conversation(first_batch['conversation'])
for k in tokenized.keys():
    tokenized[k] = torch.as_tensor(tokenized[k], dtype=torch.long)
print(tokenized['labels'].tolist())
wrapper.data_module.visualize_sample(tokenized)

In [ ]:
# mock_labels = first_batch["labels"].clone().detach()
# mock_labels = torch.where(mock_labels == -100, wrapper.data_module.tokenizer.pad_token_id, mock_labels)
# mock_ids = first_batch["input_ids"].clone().detach()
for batch in wrapper.data_module.val_dataloader():
    print(batch["input_ids"].shape)
    print(batch["labels"].shape)

In [ ]:
wrapper.init_trainer()

In [ ]:
wrapper.tokenizer

In [ ]:
wrapper.train()